In [16]:
import utils
import datetime


In [17]:
START_DATE = datetime.datetime(2024, 10, 2)
END_DATE = datetime.datetime(2025, 7, 20)
BENEFICIARY = "UMONS - NUMÉDIART"

CONVENTION_NUMBER = "???"
RESEARCHER_FIRST_NAME = "Vincent"
RESEARCHER_LAST_NAME = "Stragier"
RESEARCHER_JOB_TITLE = "assistant de recherche"
CONVENTION_OCCUPATION = 100
HOURS_PER_DAY = 7.6
FILLING_DATE = datetime.datetime.now(tz=datetime.timezone.utc)

researcher_holidays = [
    datetime.datetime(2024, 10, 2),
    datetime.datetime(2024, 10, 6),
    datetime.datetime(2024, 11, 1),
    datetime.datetime(2024, 11, 2),
]


In [18]:
years = list(range(START_DATE.year, END_DATE.year + 1))

years_and_files_to_generate = [
    (year, f"{year}___{BENEFICIARY}___{CONVENTION_NUMBER}___{RESEARCHER_LAST_NAME}_{RESEARCHER_FIRST_NAME}.xlsx".replace(" ", "_")) for year in years
]

years_and_files_to_generate


[(2024, '2024___UMONS_-_NUMÉDIART___???___Stragier_Vincent.xlsx'),
 (2025, '2025___UMONS_-_NUMÉDIART___???___Stragier_Vincent.xlsx')]

In [19]:
def get_description(
    date: datetime.datetime,
    first_day: int,
    last_day: int,
    year: int,
    researcher_holidays: list,
):
    """Generate the description for a given date.

    Args:
        date (datetime.datetime): The date to generate the description for.
        first_day (int): The first day of the month.
        last_day (int): The last day of the month.
        year (int): The year of the date.
        researcher_holidays (list): The list of researcher holidays.

    Returns:
        str: The description for the given date.
    """
    weekends, work_holidays, umons_holidays = utils.get_weekends_holidays_and_umons_holidays_for_year(
        year)

    if last_day < date.day or first_day > date.day:
        return "Hors convention", day in weekends

    if date in weekends:
        return "Fin de semaine", True

    if date in work_holidays:
        return "Jour férié", day in weekends

    if date in umons_holidays:
        return "Congé UMONS", day in weekends

    if date in researcher_holidays:
        return "Congé chercheur", day in weekends

    return "", day in weekends


content_per_file = {}

for year, file_template in years_and_files_to_generate:
    start_date = datetime.datetime(year, 1, 1)
    if year == START_DATE.year:
        start_date = START_DATE

    end_date = datetime.datetime(year, 12, 31)
    if year == END_DATE.year:
        end_date = END_DATE

    lines_in_month = {}
    for month in range(start_date.month, end_date.month + 1):
        month_str = f"{month:02d}"
        number_of_days = utils.number_of_days_in_month(year, month)

        first_day = 1
        if year == start_date.year and month == start_date.month:
            first_day = start_date.day

        last_day = number_of_days
        if year == end_date.year and month == end_date.month:
            last_day = end_date.day

        lines_in_month[month_str] = []
        for day in range(1, number_of_days + 1):
            current_date = datetime.datetime(year, month, day)
            description, day_in_weekend = get_description(
                current_date,
                first_day,
                last_day,
                year,
                researcher_holidays
            )

            work_hours = 0
            if description == "":
                work_hours = HOURS_PER_DAY

            line = {
                "day": day,
                "date": current_date.strftime("%d/%m/%Y"),
                "research_time": work_hours,
                "non_research_time": 0,
                "description": description,
                "day_in_weekend": day_in_weekend,
            }
            lines_in_month[month_str].append(line)

    content_per_file[file_template] = {
        "year": year,
        "sheets_content": lines_in_month,
        "convention_beneficiary": BENEFICIARY,
        "convention_number": CONVENTION_NUMBER,
        "researcher_first_name": RESEARCHER_FIRST_NAME,
        "researcher_last_name": RESEARCHER_LAST_NAME,
        "researcher_job_title": RESEARCHER_JOB_TITLE,
        "convention_occupation": CONVENTION_OCCUPATION,
        "hours_per_day": HOURS_PER_DAY
    }
    # print(lines_in_month)


In [21]:
# content_per_file
#

# Open the template file
import openpyxl
import pathlib
import shutil
template_file = pathlib.Path("template/template_spw.xlsx")


# template_content = openpyxl.load_workbook(
#     template_file, read_only=False, rich_text=True)

for file_template, content in content_per_file.items():
    # Copy the template file (as a file).
    destination_file = pathlib.Path("generated_timesheets") / file_template
    destination_file.parent.mkdir(parents=True, exist_ok=True)
    shutil.copy(template_file, destination_file)

    destination_workbook = openpyxl.load_workbook(
        destination_file, read_only=False, rich_text=True)

    sheets_to_keep = list(content["sheets_content"].keys())
    for sheet in destination_workbook.sheetnames:
        if sheet not in sheets_to_keep:
            destination_workbook.remove(destination_workbook[sheet])
            continue

        sheet_content = content["sheets_content"][sheet]
        sheet = destination_workbook[sheet]

        sheet["C3"].value = content["convention_beneficiary"]
        sheet["C4"].value = content["convention_number"]
        sheet["C5"].value = f"{content['researcher_first_name']} {content['researcher_last_name']}"
        sheet["D6"].value = f"{content['convention_occupation']} %"

        sheet["H3"].value = f"{content['year']}"

        for index, row in enumerate(sheet.iter_rows(min_row=39, max_row=50, min_col=1, max_col=6), start=9):
            for cell in row:
                if cell.value == "Date:":
                    sheet[f"D{cell.row}"].value = FILLING_DATE.strftime(
                        "%d/%m/%Y")

                if str(cell.value).startswith('=C5&",'):
                    cell.value = f'=C5&", {content["researcher_job_title"]}"'

        # raise Exception("STOP")

        for index, line in enumerate(sheet_content, start=9):
            sheet[f"A{index}"].value = line["day"]
            sheet[f"B{index}"].value = line["date"]
            sheet[f"C{index}"].value = line["research_time"]
            sheet[f"D{index}"].value = line["non_research_time"]
            sheet[f"F{index}"].value = line["description"]

            if line["day_in_weekend"]:
                sheet[f"F{index}"].font = openpyxl.styles.Font(
                    color="FF0000", bold=True)

            else:
                sheet[f"F{index}"].font = openpyxl.styles.Font(
                    color="000000", bold=False)

    # destination_workbook.remove(destination_workbook.active)

    # print(content)

    # sheet_to_remove = [
    #     sheet for sheet in destination_workbook.sheetnames if sheet in
    # ]

    # for sheet_name, sheet_content in content["sheets_content"].items():
    #     destination_workbook.create_sheet(sheet_name)

    #     sheet = destination_workbook[sheet_name]

    #     utils.copy_sheet_with_styles(template_content[sheet_name], sheet)

    # for line in sheet_content:
    #     sheet.append([
    #         line["day"],
    #         line["date"],
    #         line["research_time"],
    #         line["non_research_time"],
    #         line["description"],
    #     ])

    destination_workbook.save(destination_file)

#     # Save the file
#     destination_workbook.save(file_template)

# for sheet in template_content.sheetnames:
#     destination_workbook.create_sheet(sheet)


# template_content['12'].cell(row=3, column=1).value
